## Machine Learning  Sinuists

In [77]:
#Import the required libraries
import numpy as np
from keras import backend as K
np.random.seed(1338)
from PIL import Image
import cv2
import os

In [78]:
# Preprocess and Normalise Dat
img_rows, img_cols = 200, 200

input_path_t="images/resized_data/training"
input_path_v="images/resized_data/validation"

if K.image_data_format() == 'channels_first':
    shape_ord = (1, img_rows, img_cols)
else:  # channel_last
    shape_ord = (img_rows, img_cols, 3)

### Generate Dataset for Training

In [ ]:
from keras.preprocessing.image import ImageDataGenerator,img_to_array

listing = os.listdir(input_path_t)
num_samples=len(listing)

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.05,
        height_shift_range=0.05,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')
img_rows, img_cols = img_rows, img_cols


    
x = []
images=[]
files=[]
c=0
for file in listing:
    img = cv2.imread(input_path_t+"/"+file,0)  # this is a PIL image
    print(input_path_t+"/"+file)
    #print(i+ mean)
    images.append(img)
    files.append(file)
    x.append(img_to_array(img))  
    x[c] = x[c].reshape((1,) + x[c].shape)  # this is a Numpy array with shape (1, 3, 150, 150
    c=c+1


image_x=images
#create labels for training 
label=np.loadtxt('files/YT_train.csv', delimiter=',', dtype=np.int)
label_train=label
images=images[::-1]


# the .flow() command below generates batches of randomly transformed images
for a in range(0,len(images)):
    i = 0
    folder="healthy";
    if(label[a]==1):folder="unhealthy";
    #print(files[a]+" "+folder+" "+str(a))
    for batch  in datagen.flow(x[a], batch_size=1,
                              save_to_dir='images/SinusData/training/'+folder, save_prefix="sinus", save_format='png'):
        i += 1
        if i > 20:
            break  # otherwise the generator would loop indefinitely

print("Done")

## Generate Dataset for Validation

In [ ]:
from keras.preprocessing.image import ImageDataGenerator,img_to_array,load_img

datagen = ImageDataGenerator(
       rotation_range=40,
       width_shift_range=0.05,
       height_shift_range=0.05,
       rescale=1./255,
       shear_range=0.2,
       zoom_range=0.2,
       horizontal_flip=True,
       fill_mode='nearest')
img_rows, img_cols = 200, 200


listing = os.listdir(input_path_v)
num_samples=len(listing)
   
x = []
images=[]
c=0
for file in listing:
    img = cv2.imread(input_path_v+"/"+file,0)  # this is a PIL image
    images.append(img)
    x.append(img_to_array(img))  # this is a Numpy array with shape (3, 150, 150)
    x[c] = x[c].reshape((1,) + x[c].shape)  # this is a Numpy array with shape (1, 3, 150, 150
    c=c+1


# and saves the results to the `preview/` directory
label=np.loadtxt('files/YV_train.csv', delimiter=',', dtype=np.int)

# the .flow() command below generates batches of randomly transformed images
for a in range(0,len(images)):
    i = 0
    folder="healthy";
    if(label[a]==1):folder="unhealthy";
    for batch  in datagen.flow(x[a], batch_size=1,
                             save_to_dir='images/SinusData/validation/'+folder, save_prefix="sinus", save_format='png'):
        i += 1
        if i > 20:
            break  # otherwise the generator would loop indefinitely

print("Done")

In [ ]:
import cv2
import statistics
from statistics import variance

def getFeatures(image):
    height = image.shape[0]
    width = image.shape[1]
    w = 0
    b = 0
    g = 0
    for i in range(0,height):
        for j in range(0,width-1):
            if (image[i][j]>180):
                w = w+1

            elif(image[i][j]<25):
                b = b+1

    area=image.shape[0]*image.shape[1]
    g=area-(w+b)

    (means, stds) = cv2.meanStdDev(image)
    #stats = np.concatenate([means, stds]).flatten()

    ent = entropy(image).item()

    mean = np.mean(image).item()
    median = np.median(image).item()
    variance =np.var(image).item()
    SD = np.std(image, axis=None, dtype=None, out=None, ddof=0, keepdims=False).item()
    Skenss = ((mean - median)/SD)
    
    g=np.full((200,1),g)
    b=np.full((200,1),b)
    mean=np.full((200,1),mean)
    median=np.full((200,1),median)
    SD=np.full((200,1),SD)
    variance=np.full((200,1),variance)
    Skenss=np.full((200,1),Skenss)
    ent=np.full((200,1),ent)
    return g,b,mean,median,variance,Skenss,ent

def entropy(signal):
    signal = signal.ravel()
    lensig = signal.size
    symset = list(set(signal))
    propab = [np.size(signal[signal == i]) / (1.0 * lensig) for i in symset]
    ent = np.sum([p * np.log2(1.0 / p) for p in propab])
    return ent


## Preprocessing X_train,Y_train

In [ ]:
from keras import backend as K
from skimage.filters import sobel
input_path_p="images/SinusData/training/healthy"
input_path_n="images/SinusData/training/unhealthy"

listing = os.listdir(input_path_p)
dataset_size=200
num_samples=len(listing)


X_train=[]
c=0
for file in listing:
    img = cv2.imread(input_path_p+"/"+file,0)  # this is a PIL image
    #img=cv2.medianBlur(img,7)
    features=getFeatures(img)
    features=np.asarray(features)
    #features.reshape(dataset_size,-2)
    #for i in range(len(features)):img=np.hstack((img,features[i]))
    X_train.append(features)
    
listing = os.listdir(input_path_n)
len_pos=len(X_train)

for file in listing:
    img = cv2.imread(input_path_n+"/"+file,0)  # this is a PIL image
    #img=cv2.medianBlur(img,7)
    features=getFeatures(img)
    features=np.asarray(features)
    #features.reshape(dataset_size,-2)
    #for i in range(len(features)):img=np.hstack((img,features[i]))
    X_train.append(features)
    
pos_labels = np.zeros((len_pos, 1), dtype=int);  
neg_labels = np.ones((len(X_train)-len_pos, 1), dtype=int);

Y_train=np.concatenate((pos_labels, neg_labels))
X_train=np.asarray(X_train)

X_train = X_train.astype('float32')

X_train = X_train.reshape((X_train.shape[0],) + X_train[0].shape)

## Preprocessing X_test,Y_test

In [ ]:
from keras import backend as K
import numpy
input_path_p="images/SinusData/validation/healthy"
input_path_n="images/SinusData/validation/unhealthy"


listing = os.listdir(input_path_p)
num_samples=len(listing)



X_test=[]
c=0
for file in listing:
    img = cv2.imread(input_path_p+"/"+file,0)  # this is a PIL image
    #img=cv2.medianBlur(img,7)
    features=getFeatures(img)
    features=numpy.asarray(features)
    #features.reshape(dataset_size,-2)
    #for i in range(len(features)):img=np.hstack((img,features[i]))
    X_test.append(features)

    
listing = os.listdir(input_path_n)
len_pos=len(X_test)


for file in listing:
    img = cv2.imread(input_path_n+"/"+file,0)  # this is a PIL image
    #img=cv2.medianBlur(img,7)
    features=getFeatures(img)
    features=numpy.asarray(features)
    #features.reshape(dataset_size,-2)
    #for i in range(len(features)):img=np.hstack((img,features[i]))
    X_test.append(features)
    
    
pos_labels = np.zeros((len_pos, 1), dtype=int)  
neg_labels = np.ones((len(X_test)-len_pos, 1), dtype=int)
Y_test=np.concatenate((pos_labels, neg_labels))

X_test=np.asarray(X_test)
X_test = X_test.astype('float32')
X_test = X_test.reshape((X_test.shape[0],) + X_test[0].shape)

### Preprocessing X_train,X_test

In [ ]:
X_train /= 255
X_test /= 255

In [ ]:
print((X_test[1].shape))
print("Done")

### Gradinet Bossting Alogrithm Fit

In [ ]:
from sklearn.datasets import make_hastie_10_2
from sklearn.ensemble import GradientBoostingClassifier
import pickle
d=0

dataset_size = len(X_train)
Xtrain = X_train.reshape(dataset_size,-1)
#Xtrain=X_train
#Xtrain=Xtrain/255

print(Xtrain.shape)
model = GradientBoostingClassifier(n_estimators=100, learning_rate=0.1,max_depth=1, random_state=1).fit(Xtrain, np.ravel(Y_train,order='C'))

## Save Weights

In [ ]:
# save model to file
pickle.dump(model, open("features.pickle1.dat", "wb"))

## Mean Square Error And Score

In [ ]:
from sklearn.metrics import mean_squared_error,r2_score,accuracy_score
dataset_size = len(X_test)
xtest = X_test.reshape(dataset_size,-1)
mse = mean_squared_error(Y_test, model.predict(xtest))
acc = r2_score(Y_test, model.predict(xtest))

print("MSE: %.4f" % mse)
print('ACC: %.4f' % acc)


## Accuracy

In [ ]:
# predict class labels
pred = model.predict(xtest)

# score on test data (accuracy)
acc = model.score(xtest, Y_test)
print('ACC: %.4f' % acc)

## Predection Model

In [ ]:
import cv2

img_rows=200 
img_cols=200
img=cv2.imread("images/SingleCode/Labe2.png",0);
img = cv2.resize(img, (img_rows, img_cols), interpolation = cv2.INTER_CUBIC)
print(img.shape)
print(img.shape)
features=getFeatures(img)
features=numpy.asarray(features)
h,u=model.predict_proba(features)[0]
if(h>u):print("Healthy",h,u);
else:print("UnHelathy",h,u);

## Read Weight from File and Predict 

In [95]:
# load model from file
import pickle
import cv2
import numpy

img_rows, img_cols= 200,200

loaded_model = pickle.load(open("features.pickle1.dat", "rb"))
    
img=cv2.imread("images/Portal/sinus_left5.png");
img = cv2.resize(img, (img_rows, img_cols), interpolation = cv2.INTER_CUBIC)
print("Before: ",len(img[0]))
img=img.reshape(1,-1)
features=getFeatures(img)
features=numpy.asarray(features)
print("features",features.shape)
print("After: ",len(img[0]))
h,u=loaded_model.predict_proba(features)[0]
if(h>u):print("Healthy",h,u);
else:print("UnHelathy",h,u);

Before:  200
features (7, 200, 1)
features (1, 1400)
After:  120000
UnHelathy 0.24445836826396683 0.7555416317360332


In [ ]:
import matplotlib.pyplot as plt

params = {'n_estimators': 100, 'max_depth': 1,
        'learning_rate': 1, 'loss': 'huber','alpha':0.95}
test_score = np.zeros((params['n_estimators'],), dtype=np.float64)

for i, y_pred in enumerate(model.staged_decision_function(xtest)):
    test_score[i] = model.loss_(Y_test, y_pred)
    #print(test_score)

plt.figure(figsize=(12, 6))
plt.subplot(1, 1, 1)
plt.title('Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, model.train_score_, 'b-',
                label='Training Set Deviance')
plt.plot(np.arange(params['n_estimators']) + 1, test_score, 'r-',
                label='Test Set Deviance')
plt.legend(loc='upper right')
plt.xlabel('Boosting Iterations')
plt.ylabel('Deviance')